In [149]:
import pandas as pd
from pathlib import Path
import json

#https://medium.com/swlh/converting-nested-json-structures-to-pandas-dataframes-e8106c59976e
#https://github.com/MrFuguDataScience/JSON/blob/master/Nested_Json_Mult_Ex.ipynb
#https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.io.json.build_table_schema.html

In [161]:
with open("SampleRoutingData.json") as f:
    data =json.load(f)

In [162]:
df = pd.json_normalize(data)

df = df.explode('Body')

df.items()

<generator object DataFrame.items at 0x7fdbfd390650>

In [163]:
df=pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in df.items() ]))

df=pd.json_normalize(json.loads(df.to_json(orient="records")))
df=df.explode('Body.payload')

df = pd.concat([df.drop(['Body.payload'], axis=1), df['Body.payload'].apply(pd.Series)], axis=1)

In [164]:
print(df.shape)
print(df.head(2))

(8, 8)
   EventProperties.EnquedMessageTime EventProperties.MessageType  \
0                      1607243131479               IoTHubRouting   
0                      1607243131479               IoTHubRouting   

     Body.msgtype  Body.Timestamp  ID   Sensor  Value     Unit  
0  TELEMETRY DATA   1607243134759   1     Temp   30.0  Celsius  
0  TELEMETRY DATA   1607243134759   2  Current   58.0     Volt  
